In [3]:
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random


from .utils import *
from .constants import *

# connect to the sim 
client = carla.Client('localhost', 2000)

time.sleep(5)
client.set_timeout(25)

world = client.get_world()

# ensure sync mode on 
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1
settings.no_rendering_mode = False
world.apply_settings(settings)

spawn_points = world.get_map().get_spawn_points()

#clean up any existing cars
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

#look for a blueprint of Tesla m3 car
vehicle_bp = world.get_blueprint_library().filter('*model3*')

def exit_clean():
    #clean up
    cv2.destroyAllWindows()
    camera_sem.stop()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    return None

data_path = '../data/automatic-navigation/segmentation'

#main loop
img_counter = 0
quit = False
while img_counter < 10000:
    start_point = random.choice(spawn_points)
    vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
    time.sleep(2)
    # setting semantic camera
    camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
    camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.13 on Windows
    camera_bp.set_attribute('image_size_y', '480')
    camera_bp.set_attribute('fov', '90')
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    camera_sem = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = 640
    image_h = 480

    camera_data = {'sem_image': np.zeros((image_h,image_w,4)),
                   'rgb_image': np.zeros((image_h,image_w,4))}

        # this actually opens a live stream from the cameras
    camera_sem.listen(lambda image: sem_callback(image,camera_data))
    # adding collision sensor
    
    collision_data ={'collision': False}
    
    collision_bp = world.get_blueprint_library().find('sensor.other.collision')
    collision_sensor = world.spawn_actor(collision_bp,carla.Transform(), attach_to = vehicle)
    collision_sensor.listen(lambda event: collision_callback(event,collision_data))    
    
    prev_position = vehicle.get_transform()
    # getting a random route for the car
    route = select_random_route(start_point,spawn_points)
    curr_wp = 5 #we will be tracking waypoints in the route and switch to next one when we get close to current one
    predicted_angle = 0
    PREFERRED_SPEED = 40 # setting speed at start of new route
    
    while curr_wp<len(route)-1 and not collision_data['collision']:
        # Carla Tick
        world.tick()
        
        if curr_wp >=len(route)-5: # within 10 points of end, the route is done
            exit_clean()
            break
        while curr_wp<len(route)-2 and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location)<5:
            curr_wp +=1 #move to next wp if we are too close
        curr_wp, predicted_angle = get_proper_angle(vehicle,curr_wp,route)
        gen_dir_angle = get_distant_angle(vehicle,curr_wp,route,30)
        
        v = vehicle.get_velocity()
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
        estimated_throttle = maintain_speed(speed)

        steer_input = predicted_angle
        # limit steering to max angel, say 40 degrees
        if predicted_angle<-MAX_STEER_DEGREES:
            steer_input = -MAX_STEER_DEGREES
        elif predicted_angle>MAX_STEER_DEGREES:
            steer_input = MAX_STEER_DEGREES
        # conversion from degrees to -1 to +1 input for apply control function is applied below in steering input
        current_position = vehicle.get_transform()
        sem_im = camera_data['sem_image']
        vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input/STEERING_CONVERSION))
        #write images
        if current_position.location.distance(prev_position.location)>5:
            prev_position = current_position
            img_counter += 1
            time_grab = time.time_ns()
            cv2.imwrite(f'{data_path}/%06d_%s_%s.png' % (time_grab, gen_dir_angle,round(predicted_angle,0)), sem_im)
    
    if quit:
        break
exit_clean()

# Add some noise

In [ ]:
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random

from .utils import *
from .constants import *

client = carla.Client('localhost', 2000)
time.sleep(5)
client.set_timeout(25)

# get world and spawn points
world = client.get_world()

 
settings = world.get_settings()
settings.synchronous_mode = False
if settings.synchronous_mode:
    settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

cleanup(world=world)
spawn_points = world.get_map().get_spawn_points()

quit = False
while not quit:
    start_point = random.choice(spawn_points)
    vehicle_bp = world.get_blueprint_library().filter('*model3*')
    vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
    time.sleep(2)
 
    #RGB CAM
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
    camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
    camera_bp.set_attribute('fov', str(FOV))
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    #this creates the camera in the sim
    camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = camera_bp.get_attribute('image_size_x').as_int()
    image_h = camera_bp.get_attribute('image_size_y').as_int()
    camera_data = {'image': np.zeros((image_h,image_w,3)),
                'sem_image': np.zeros((image_h,image_w,3))}
    # this actually opens a live stream from the camera
    camera.listen(lambda image: camera_callback(image,camera_data))

    #Semantgic cam
    sem_camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
    sem_camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
    sem_camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
    sem_camera_bp.set_attribute('fov', str(FOV))
    #this creates the camera in the sim
    sem_camera = world.spawn_actor(sem_camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = sem_camera_bp.get_attribute('image_size_x').as_int()
    image_h = sem_camera_bp.get_attribute('image_size_y').as_int()
    sem_camera_data = {'image': np.zeros((image_h,image_w,3))}
    # this actually opens a live stream from the camera
    #sem_camera.listen(lambda image: camera_callback(image,sem_camera_data))
    sem_camera.listen(lambda image: sem_callback(image,camera_data))

    cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
    cv2.imshow('RGB Camera',camera_data['image'])
    #make a route
    route = select_random_route(start_point,spawn_points)
    for idx, waypoint in enumerate(route): # move the car through the route
        #world.tick()
        transform = waypoint[0].transform
        vehicle.set_transform(transform)
        vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
        time.sleep(2) #these delays seem to be necessary for teh car to take the position before a shot is taken
        initial_yaw = waypoint[0].transform.rotation.yaw
        
        for i in range(5):
            #world.tick()         
            trans = waypoint[0].transform
            angle_adj = random.randrange(-YAW_ADJ_DEGREES, YAW_ADJ_DEGREES, 1)
            trans.rotation.yaw = initial_yaw +angle_adj 
            vehicle.set_transform(trans)
            vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
            time.sleep(1)  #these delays seem to be necessary for the car to take the position before a shot is taken
            gen_dir_angle = get_distant_angle(vehicle,idx,route,30) # general angle taken before spinning the car
            if cv2.waitKey(1) == ord('q'):
                quit = True
                break
            # Display with imshow
            cv2.imshow('RGB Camera',camera_data['image'])
            # save semantic image watching out for end of route    
            if idx +5 < len(route)-1:
                predicted_angle = get_angle(vehicle,route[idx+5][0]) # we always get the angle to +5 waypoint ahead of us
                
                time_grab = time.time_ns()
                sem_image = camera_data['sem_image']
                if np.sum(sem_image) > 0:   #check for black images
                    cv2.imwrite('_img/%06d_%s_%s.png' % (time_grab, gen_dir_angle,round(predicted_angle,0)), sem_image)
    cleanup(world=world)
    if quit:
        break
    # Break loop if user presses q
cv2.destroyAllWindows()
cleanup(world=world)


In [ ]:
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random

from .utils import *
from .constants import *

client = carla.Client('localhost', 2000)
time.sleep(5)
client.set_timeout(25)

# get world and spawn points
world = client.get_world()

 
settings = world.get_settings()
settings.synchronous_mode = False
if settings.synchronous_mode:
    settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

cleanup(world=world)
spawn_points = world.get_map().get_spawn_points()

quit = False
while not quit:
    start_point = random.choice(spawn_points)
    vehicle_bp = world.get_blueprint_library().filter('*model3*')
    vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
    time.sleep(2)
 
    #RGB CAM
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
    camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
    camera_bp.set_attribute('fov', str(FOV))
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    #this creates the camera in the sim
    camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = camera_bp.get_attribute('image_size_x').as_int()
    image_h = camera_bp.get_attribute('image_size_y').as_int()
    camera_data = {'image': np.zeros((image_h,image_w,3)),
                'sem_image': np.zeros((image_h,image_w,3))}
    # this actually opens a live stream from the camera
    camera.listen(lambda image: camera_callback(image,camera_data))

    #Semantgic cam
    sem_camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
    sem_camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
    sem_camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
    sem_camera_bp.set_attribute('fov', str(FOV))
    #this creates the camera in the sim
    sem_camera = world.spawn_actor(sem_camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = sem_camera_bp.get_attribute('image_size_x').as_int()
    image_h = sem_camera_bp.get_attribute('image_size_y').as_int()
    sem_camera_data = {'image': np.zeros((image_h,image_w,3))}
    # this actually opens a live stream from the camera
    #sem_camera.listen(lambda image: camera_callback(image,sem_camera_data))
    sem_camera.listen(lambda image: sem_callback(image,camera_data))

    cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
    cv2.imshow('RGB Camera',camera_data['image'])
    #make a route
    route = select_random_route(start_point,spawn_points)
    gen_dir_angle = 0 # in case we did not get a general GPS direction
    for idx, waypoint in enumerate(route): # move the car through the route
        
        transform = waypoint[0].transform
        vehicle.set_transform(transform)
        vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
        time.sleep(2) #these delays seem to be necessary for teh car to take the position before a shot is taken
        initial_yaw = waypoint[0].transform.rotation.yaw
        # GPS general direction is only taken outside intersections
        # so that direction is retained throughout the same intersection until it is finished
        if not waypoint[0].is_intersection and not waypoint[0].is_junction:
            gen_dir_angle = get_distant_angle(vehicle,idx,route,30) # general angle taken before spinning the car
        # logic to detect a lane change and ignore/not take those images
        draw_route(world, idx, route,seconds=5.0)
        lane_change = False
        if not waypoint[0].is_intersection and not waypoint[0].is_junction:
            if idx < len(route)-2:
                if route[idx][0].lane_id != route[idx+1][0].lane_id:
                    lane_change = True
        if not lane_change:
            for i in range(5):
                trans = waypoint[0].transform
                angle_adj = random.randrange(-YAW_ADJ_DEGREES, YAW_ADJ_DEGREES, 1)
                trans.rotation.yaw = initial_yaw +angle_adj 
                vehicle.set_transform(trans)
                vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
                time.sleep(1)  #these delays seem to be necessary for the car to take the position before a shot is taken
                
                if cv2.waitKey(1) == ord('q'):
                    quit = True
                    break
                # Display with imshow
                cv2.imshow('RGB Camera',camera_data['image'])
                # save semantic image watching out for end of route    
                if idx +5 < len(route)-1:
                    predicted_angle = get_angle(vehicle,route[idx+5][0]) # we always get the angle to +5 waypoint ahead of us
                    
                    time_grab = time.time_ns()
                    sem_image = camera_data['sem_image']
                    if np.sum(sem_image) > 0:   #check for black images
                        cv2.imwrite('_img/%06d_%s_%s.png' % (time_grab, gen_dir_angle,round(predicted_angle,0)), sem_image)
    cleanup(world=world)
    if quit:
        break
    # Break loop if user presses q
cv2.destroyAllWindows()
cleanup(world=world)